In [2]:
import pandas as pd
from models.utils.sp_scraper import SPScraper
import yfinance as yf
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow import keras
import talib as ta
import time
from IPython.display import clear_output
from concurrent.futures import ThreadPoolExecutor, as_completed
import joblib
import os

In [3]:
model = keras.models.load_model('/Users/aryanhazra/Downloads/VSCode Repos/trading_model/src/models/model3/model3.keras')

# Load all scalers
scaler_future_price = joblib.load('/Users/aryanhazra/Downloads/VSCode Repos/trading_model/src/models/model3/scaler_future_price.pkl')
scaler_ticker = joblib.load('/Users/aryanhazra/Downloads/VSCode Repos/trading_model/src/models/model3/scaler_ticker.pkl')
scaler_technical = joblib.load('/Users/aryanhazra/Downloads/VSCode Repos/trading_model/src/models/model3/scaler_technical.pkl')

# Initialize the scalers dictionary
scalers = {}

# Path to the scalers directory
scalers_dir = '/Users/aryanhazra/Downloads/VSCode Repos/trading_model/src/models/model3/scalers'

# Loop through each ticker directory
for ticker in os.listdir(scalers_dir):
    ticker_path = os.path.join(scalers_dir, ticker)
    if os.path.isdir(ticker_path):
        # Load all three scalers for this ticker
        scaler_close = joblib.load(os.path.join(ticker_path, 'scaler_close.pkl'))
        scaler_sma_10 = joblib.load(os.path.join(ticker_path, 'scaler_sma_10.pkl'))
        scaler_sma_30 = joblib.load(os.path.join(ticker_path, 'scaler_sma_30.pkl'))
        
        # Store them in a dictionary
        scalers[ticker] = {
            'scaler_close': scaler_close,
            'scaler_sma_10': scaler_sma_10,
            'scaler_sma_30': scaler_sma_30
        }

2025-04-13 20:59:12.713435: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-04-13 20:59:12.713463: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-04-13 20:59:12.713468: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
I0000 00:00:1744592352.713478  343581 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1744592352.713498  343581 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Replace '.' with '-' in ticker symbols, also add SPY as a benchmark
scraper = SPScraper()
sp_tickers = [ticker.replace(".", "-") for ticker in sorted(scraper.scrape_sp500_symbols().index)] + ["^GSPC"]


data_frames = []

def process_ticker(ticker):
    try:
        # Retry logic
        while True:
            try:
                ticker_data = yf.Ticker(ticker).history(period="1y")
                break
            except Exception:
                time.sleep(10)

        if ticker_data.empty:
            return None

        # Process data
        ticker_data.reset_index(inplace=True)
        ticker_data.columns = ticker_data.columns.str.lower()
        ticker_data['ticker'] = ticker

        ticker_data['rsi'] = ta.RSI(ticker_data['close'], timeperiod=14)
        macd, macdsignal, macdhist = ta.MACD(ticker_data['close'], fastperiod=12, slowperiod=26, signalperiod=9)
        ticker_data['macd'] = macd
        ticker_data['sma_10'] = ta.SMA(ticker_data['close'], timeperiod=10)
        ticker_data['sma_30'] = ta.SMA(ticker_data['close'], timeperiod=30)

        # Initialize scalers
        scaler_close = StandardScaler()
        scaler_future_price = StandardScaler()
        scaler_sma_10 = StandardScaler()
        scaler_sma_30 = StandardScaler()

        # Select and scale
        close_vals = ticker_data[['close']].values
        sma_10_vals = ticker_data[['sma_10']].values
        sma_30_vals = ticker_data[['sma_30']].values

        ticker_data['scaled_close'] = scalers[ticker]['scaler_close'].transform(close_vals)
        ticker_data['scaled_sma_10'] = scalers[ticker]['scaler_sma_10'].transform(sma_10_vals)
        ticker_data['scaled_sma_30'] = scalers[ticker]['scaler_sma_30'].transform(sma_30_vals)

        # Save scalers
        ticker_scalers = {
            'scaler_close': scaler_close,
            'scaler_sma_10': scaler_sma_10,
            'scaler_sma_30': scaler_sma_30
        }

        return ticker_data
    
    except Exception as e:
        print(f"Failed to process {ticker}: {e}")
        return None

# Use multithreading for I/O-bound operations like data download
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(process_ticker, ticker): ticker for ticker in sp_tickers}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading data", unit="ticker"):
        result_data = future.result()
        if result_data is not None:
            data_frames.append(result_data)
        else:
            print(f"Failed to process {future.result()}")

# Combine all dataframes
data = pd.concat(data_frames, ignore_index=True)
data.dropna(inplace=True)

In [5]:
# Prepare the data for the model
# Label encode the ticker column
label_encoder = LabelEncoder()
data["encoded_ticker"] = label_encoder.fit_transform(data["ticker"])

# Initialize scalers
scaler_ticker = StandardScaler()
scaler_technical = StandardScaler()


# Scale the ticker column
stock_ticker = data.filter(["encoded_ticker"])
stock_ticker = stock_ticker.values
scaled_ticker = scaler_ticker.fit_transform(stock_ticker)
data['scaled_ticker'] = scaled_ticker

#scale technical columns
stock_technical = data.filter(["return", "rsi", "macd"])
stock_technical = stock_technical.values
scaled_technicals = scaler_technical.fit_transform(stock_technical)
# Insert scaled data into the original dataframe
data['scaled_rsi'] = scaled_technicals[:, 0]
data['scaled_macd'] = scaled_technicals[:, 1]

# Group the data by ticker
grouped_dfs = data.groupby('ticker')
grouped_dfs = {ticker: df.sort_values(by='date').reset_index(drop=True) for ticker, df in grouped_dfs}

In [6]:
feature_cols = ['scaled_close', 'scaled_rsi', 'scaled_macd', 'scaled_sma_10', 'scaled_sma_30', 'scaled_ticker']

#take curr day, find most recent day in each df
today = pd.Timestamp.now().strftime('%Y-%m-%d')
x_pred = []
for ticker, df in grouped_dfs.items():
    most_recent_day = df['date'].max()
    most_recent_index = df[df['date'] == most_recent_day].index[0]

    window = df.iloc[most_recent_index - 60:][feature_cols].values

    x_pred.append([ticker,window])

#predicting based on the second vals of x_test (the windows)
predictions = model.predict(np.array([x[1] for x in x_pred]))
#inverting the scaling
predictions = scaler_future_price.inverse_transform(predictions)
# Assigning predictions, and real vals based on ticker
predictions = {x_pred[i][0]: (float(predictions[i][0])) for i in range(len(predictions))}
#sorting the predictions by the first value (the predicted vals)
top_10_predictions = dict(sorted(predictions.items(), 
                                key=lambda x: x[1], 
                                reverse=True)[:10])
clear_output(wait=True)  # The wait=True parameter prevents flickering
print(f"\nPredictions for date: {most_recent_day}")
print("Top 10 Predicted Returns:")
print("Ticker | Predicted Return")
print("-" * 30)
for ticker, pred in top_10_predictions.items():
# Convert to percentages by multiplying by 100
    print(f"{ticker:6} | {pred*100:13.2f}%")




Predictions for date: 2025-04-11 00:00:00-04:00
Top 10 Predicted Returns:
Ticker | Predicted Return
------------------------------
TXT    |         16.06%
PPG    |         15.21%
TGT    |         15.15%
OXY    |         14.62%
WDC    |         14.58%
ON     |         14.40%
TMO    |         14.06%
SWKS   |         13.98%
MRK    |         13.80%
FANG   |         13.78%
